In [6]:
import os
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage

In [9]:
load_dotenv()

# Setelah dimuat, AMBIL nilainya ke dalam sebuah variabel Python
google_api_key = os.getenv("GOOGLE_API_KEY")

# Lakukan pengecekan untuk memastikan key-nya ada
if google_api_key:
    print("✅ API Key berhasil dimuat dari file .env.")
    # Set environment variable untuk digunakan oleh library lain seperti LangChain
    os.environ["GOOGLE_API_KEY"] = google_api_key
else:
    print("❌ ERROR: Tidak dapat menemukan GOOGLE_API_KEY di dalam file .env.")

✅ API Key berhasil dimuat dari file .env.


In [11]:
# Langkah 1: Definisikan Tool untuk Kalkulasi

class ManpowerRecommendationInput(BaseModel):
    """Skema input untuk tools rekomendasi kebutuhan manpower kebersihan disebuah fasilitas gedung perkantoran."""
    luas_lantai_m2: int = Field(description="Total luas lantai sebuah fasilitas dalam satuan meter persegi.")
    jumlah_toilet: int = Field(description="Total jumlah toilet yang ada di dalam fasilitas tersebut.")

@tool(args_schema=ManpowerRecommendationInput)
def rekomendasi_jumlah_manpower(luas_lantai_m2: int, jumlah_toilet: int) -> dict:
    """
    Gunakan toole ini untuk menghitung rekomendasi jumlah manpower (supervisor, team leader, admin, gardener, external, cleaning service, OB) yang dibutuhkan disebuah fasilitas berdasarkan luas dan jumlah toilet.
    """
    print(f"\[Sistem] === Tool 'rekomendasi_jumlah_manpower' dieksekusi ===")
    print(f"[Sistem] Menerima data: Luas = {luas_lantai_m2} m2, Toilet = {jumlah_toilet}")

    prediksi = (luas_lantai_m2/1000) + (jumlah_toilet/10)
    hasil_akhir = round(prediksi)

    print(f"[Sistem] Hasil kalkulasi: {hasil_akhir} orang.")
    print("[Sistem ===================================\n]")

    return{"rekomendasi_manpower": hasil_akhir}

In [ ]:
# Langkah 2: Membuat Agent yang Akan Menggunakan Tool

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest", temperature=0)
tools = [rekomendasi_jumlah_manpower]

prompt = ChatPromptTemplate.from_messages([
("system","""
 Anda adalah "Hoffbot", AI asisten yang cerdas dan ramah dari PT. Hoffmen Cleanindo.
 Tujuan utama Anda adalah membatu klien mendapatkan estimasi kebutuhan manpower.
 
 ALUR KERJA:
 1.Menyapa klien dan pahami permintaan mereka.
 2. Untuk menjawab permintaan, Anda HARUS menggunakan tool "rekomendasi_jumlah_manpower.
 3. Jika data yang dibutuhkan oleh tool (luas dan jumlah toilet) belum lengkap, tanyakan kepada klien untuk informasi yang kurang tersebut. JANGAN membuat asumsi.
 4. Setelah tool berhasil dieksekusi, sampaikan hasilnya kepada klien dalam format laporan yang profesional dan mudah dibaca.
 5. Anda juga bisa memberikan 1-2 saran tambahan yang relevan setelah memberikan hasil rekomendasi utama.
"""),
MessagesPlaceholder(variable_name="chat_history"),
("user", "{input}"),
MessagesPlaceholder(variable_name="agent_scratchpad"),
 ])

agent = create_tool_calling_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)


In [16]:
# Langkah 3: Membuat Loop Percakapan Interaktif

if __name__ == "__main__":
    print("Hoffbot: Halo, saya Hoffbot, asistem AI dari PT. Hoffmen Cleanindo. Ada yang bisa saya bantu terkait perencanaan manpower Anda?")

    chat_history = []

    while True:
        try:
            user_input = input("Anda:")
            if user_input.lower() in ["exit", "quit", "keluar"]:
                print("Hoffbot: Terima kasih telah menggunakan layanan kami. Sampai jumpa!")
                break
            
            result = agent_executor.invoke({
                "input": user_input,
                "chat_history":chat_history
            })

            chat_history.extend([
                HumanMessage(content = user_input),
                AIMessage(content=result["output"]),
            ])
        except Exception as e:
            print(f"Terjadi error: {e}")

Hoffbot: Halo, saya Hoffbot, asistem AI dari PT. Hoffmen Cleanindo. Ada yang bisa saya bantu terkait perencanaan manpower Anda?


> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [5]:
import os
import math
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage

load_dotenv()

# Setelah dimuat, AMBIL nilainya ke dalam sebuah variabel Python
google_api_key = os.getenv("GOOGLE_API_KEY")

# Lakukan pengecekan untuk memastikan key-nya ada
if google_api_key:
    print("✅ API Key berhasil dimuat dari file .env.")
    # Set environment variable untuk digunakan oleh library lain seperti LangChain
    os.environ["GOOGLE_API_KEY"] = google_api_key
else:
    print("❌ ERROR: Tidak dapat menemukan GOOGLE_API_KEY di dalam file .env.")

# Langkah 1: Definisikan Tool untuk Kalkulasi

class ManpowerRecommendationInput(BaseModel):
    tipe_properti: str = Field(description="Tipe properti, harus salah satu dari: 'Gedung Kantor', 'Apartemen', 'Mall', 'Rumah Sakit'.")
    luas_total_area: int = Field(description="Total luas area yang aktif digunakan dalam satuan meter persegi/m².")
    jumlah_lobby: int = Field(description="Jumlah lobi utama di dalam gedung.", default= 1)
    jumlah_lantai: int = Field(description="Jumlah lantai yang ada di dalam gedung.")
    jumlah_kamar_mandi_pria: int = Field(description = "Jumlah kamar mandi pria yang ada di dalam gedung.")
    jumlah_kamar_mandi_wanita: int = Field(description = "Jumlah kamar mandi wanita yang ada di dalam gedung.")
    jumlah_gondola: int = Field(description = "Jumlah gondola yang ada di dalam gedung.", default=0)
    luas_area_hijau: int = Field(desciption = "Total luas area taman/hijau dalam meter persegi/m²")
    kapasitas_pengunjung: [int] = Field(None, description = "Estimasi kapasitas pengunjungharian. Wajib untuk Mall")
    jumlah_area_publik_apartemen: [int] = Field(None, description = "Jumlah lantai dengan fasilitas umum (lobi, kolam renang, gym). WAJIB untuk Apartemen.")

@tool(args_schema=ManpowerRecommendationInput)
def rekomendasi_jumlah_manpower(**kwargs) -> dict:
    """
    Gunakan tools ini untuk menghitung rekomendasi jumlah manpower (supervisor, team leader, admin, gardener, cleaning service, OB) yang dibutuhkan disebuah fasilitas.
    """
    print(f"\[Sistem] === Tool 'rekomendasi_jumlah_manpower' dieksekusi ===")

    tipe_properti = kwargs.get("tipe_properti")
    jumlah_cso = 0
    catatan_tambahan = []

    if tipe_properti in ["Gedung Kantor"]:
        jumlah_lantai = kwargs.get("jumlah_lantai") 
        if not jumlah_lantai: return {"error": "Untuk Office Tower, jumlah lantai wajib diisi."}
        jumlah_cso = math.ceil(jumlah_lantai /8) *2

    elif tipe_properti == "mall":
        luas_total_area = kwargs.get("luas_total_area")
        kapasitas = kwargs.get("kapasitas_pengunjung")
        if not luas_total_area or not kapasitas: return {"error":"Untuk Mall, luas area dan kapasitas pengunjung wajib diisi."}
        # Aturan dummy: 1 CSO per 1500 m2 + tambahan untuk toilet berdasarkan kapasitas
        cso_area = math.ceil(luas_total_area/1500)
        cso_toilet = 0
        if kapasitas <=1000:
            cso_toilet = 4 # 2 couple
        else:
            cso_toilet = 8 # 4 couple
        total_cso = cso_area + cso_toilet

    elif tipe_properti == "Apartemen":
        luas_total_area = kwargs.get("luas_total_area")
        jumlah_area_publik_apartemen = kwargs.get("jumlah_area_publik_apartemen")
        if not luas_total_area or not jumlah_area_publik_apartemen: return{"error": "Untuk Apartemen, luas area dan jumlah lantai publik wajib diisi."}
        # Aturan dummy: 1 CSP per 2500m2 + 1 couple per lantai publik
        cso_area = math.ceil(luas_total_area/2500)
        cso_toilet_umum = jumlah_area_publik_apartemen * 2
        total_cso = cso_area + cso_toilet_umum

    elif tipe_properti == "Rumah Sakit":
        luas_total_area = kwargs.get("luas_total_area")
        if not luas_total_area: return {"error": "Untuk Rumah Sakit, luas area wajib diisi"}
        # Aturan dummy: 1 CSO per 500m2 karena standar higineis tinggi
        total_cso = math.ceil(luas_total_area/500)
    
    else:
        return {"error": f"Tipe properti '{tipe_properti}'' tidak dikenal."}
               
    # Hitung Supervisi TL & SPV)
    total_tl = math.ceil(total_cso/10)
    total_spv = 1

    # Perhitungan gardener
    luas_area_hijau = kwargs.get("luas_area_hijau", 0)
    jumlah_gardener = 0
    if luas_area_hijau > 0:
        ratio_gardener = 4500 #Default
        if tipe_properti == "Rumah Sakit": ratio_gardener = 2500
        elif tipe_properti == "Mall": ratio_gardener = 3250
        elif tipe_properti == "Office Tower": ratio_gardener = 4000
        jumlah_gardener = math.ceil(luas_area_hijau/ratio_gardener)

    # Perhitungan Gondola
    jumlah_unit_gondola = kwargs.get("jumlah_gondola", 0)
    jumlah_operator_gondola = jumlah_unit_gondola * 3

    # Hitung subtotal staf operasional terlebih dahulu
    sub_total_operasional = total_cso + total_tl + total_spv + jumlah_gardener + jumlah_operator_gondola

    # Perhitungan Admin
    jumlah_admin = 0
    if sub_total_operasional > 30:
        jumlah_admin = 1
        catatan_tambahan.append("1 Admin ditambahkan karena total tim operasional lebih dari 30 orang.")

    # Kalkulasi Total Keseluruhan
    total_manpower = sub_total_operasional + jumlah_admin

    hasil = {
        "estimasi_total_manpower": total_manpower,
        "rincian":{
            "CSO (Cleaning Service Officer)": total_cso,
            "Supervisor": total_spv,
            "Team Leader": total_tl,
            "Admin": jumlah_admin,
            "Gardener": jumlah_gardener,
            "Operator Gondola": jumlah_operator_gondola,
        },
        "catatan_tambahan": catatan_tambahan
    }
    print(f"[Sistem] Hasil kalkulasi:{hasil}")
    print("[Sistem]=================================================\n")
    return hasil

# Langkah 2: Membuat Agent yang Akan Menggunakan Tool

SYSTEM_PROMPT= """
 Anda adalah "Hoffbot", AI asisten yang cerdas dan ramah dari PT. Hoffmen Cleanindo.
 Tujuan utama Anda adalah membatu klien mendapatkan estimasi kebutuhan manpower.
 
ALUR KERJA:
1.  Sapa pengguna, lalu selalu tanyakan **Tipe Properti** terlebih dahulu.
2.  Berdasarkan Tipe Properti, tanyakan parameter utamanya (Jumlah Lantai untuk Office Tower, atau Luas Area untuk yang lain, serta parameter spesifik lainnya seperti kapasitas atau area publik).
3.  Setelah itu, tanyakan secara proaktif:
    - "Apakah ada area taman/hijau yang perlu dirawat? Jika ya, berapa perkiraan luasnya dalam meter persegi?"
    - "Apakah gedung dilengkapi dengan unit gondola? Jika ya, ada berapa unit?"
4.  Kumpulkan semua informasi yang relevan, lalu panggil tool `estimasi_manpower_lengkap`.
5.  Sajikan hasilnya dalam format rincian yang profesional, termasuk jika ada catatan tambahan dari hasil perhitungan.
"""

# Langkah 3: Membuat Loop Percakapan Interaktif

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest", temperature=0)
tools = [rekomendasi_jumlah_manpower]
prompt = ChatPromptTemplate.from_messages([("system", SYSTEM_PROMPT), MessagesPlaceholder(variable_name = "chat_history"), ("user", "{input}"), MessagesPlaceholder(variable_name="agent_scratchpad")])
agent = create_tool_calling_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)



if __name__ == "__main__":
    print("Hoffbot: Halo, saya Hoffbot, asistem AI dari PT. Hoffmen Cleanindo. Saya akan membantu Anda menghitung kebutuhan manpower untuk timm cleaning secara lengkap.")
    chat_history = []
    while True:
        user_input = input("Anda:")
        if user_input.lower() in ["exit", "quit", "keluar"]:
            break
        result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
        chat_history.extend([HumanMessage(content = user_input), AIMessage(content=result["output"])])


c:\Users\ASUS\.pyenv\pyenv-win\versions\3.9.13\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ API Key berhasil dimuat dari file .env.


PydanticSchemaGenerationError: Unable to generate pydantic-core schema for [<class 'int'>]. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.11/u/schema-for-unknown-type

In [ ]:
    # else: # Untuk Mall, Apartemen, Rumah Sakit, dll.
    #     luas_area = kwargs.get('luas_total_area') 
    #     if not luas_area: return {"error": f"Untuk {tipe_properti}, luas area wajib diisi."}
    #     ratio_luas = 1500 # Default                       
    #     if tipe_properti == 'Rumah Sakit': ratio_luas = 500
    #     elif tipe_properti == 'Apartemen': ratio_luas = 2500
    #     total_cso = math.ceil(luas_area / ratio_luas)

    elif tipe_properti == "Mall":
        luas_area_mall = kwargs.get("luas_total_area")
        jumlah_lantai_mall = kwargs.get("jumlah_lantai")
        jumlah_toilet_couple_mall = kwargs.get("jumlah_toilet_couple") * 2
        jumlah_toilet_single_mall = kwargs.get("jumlah_toilet_single")
        jumlah_shift_mall = kwargs.get("jumlah_shift")
        if not luas_area_mall or not jumlah_lantai_mall:
            return{"error", "Untuk Mall, luas area dan jumlah lantai wajib diisi."}
        
        # Kapasitas kerja per CS
        kapasitas_area_umum_mall_per_cso = 1500
        kapasitas_toilet_couple_per_cso = 6
        kapasitas_parkir_per_cso = 3000

        # Hitung jumlah CS per Kategori
        cso_area_umum = math.ceil(luas_area_mall / kapasitas_area_umum_mall_per_cso)
        cso_toilet = math.ceil((jumlah_toilet_couple_mall + jumlah_toilet_single_mall)/kapasitas_toilet_couple_per_cso) if jumlah_toilet_couple_mall else 0
       
        cso_toilet_standby = 0
        if trafik_padat and jumlah_toilet_couple_mall:
            cso_toilet_standby = math.ceil(jumlah_toilet_couple_mall/20)

        total_cso_per_shift = cso_area_umum + cso_toilet + cso_toilet_standby
        total_cs = total_cso_per_shift * jumlah_shift_mall